In [25]:
import os

from PIL import Image, ImageEnhance, ImageFilter #image enhance and filter just for the synthetic noising

import numpy as np
import random

import cv2 # for synthetically noising

from matplotlib import pyplot as plt #for visualizing 

### Sub functions for distorting images

In [26]:
def add_color_cast(np_image):
    blue_tint = random.uniform(0.9, 2) #changed
    green_tint = random.uniform(0.9, 2) #changed
    red_tint = random.uniform(0.25, 1.0) #changed
    np_image[:, :, 0] *= blue_tint
    np_image[:, :, 1] *= green_tint 
    np_image[:, :, 2] *= red_tint
    return np.clip(np_image, 0, 255)

In [27]:
def add_gaussian_blur(image):
    blur_radius = random.uniform(3, 5) #changed
    return image.filter(ImageFilter.GaussianBlur(radius=blur_radius))

In [28]:
def add_gaussian_noise(np_image):
    noise_std = random.uniform(20, 60)
    noise = np.random.normal(0, noise_std, np_image.shape).astype(np.float32)
    return np.clip(np_image + noise, 0, 255)

In [29]:
def adjust_brightness(image):
    brightness_factor = random.uniform(0.5, 2)
    enhancer = ImageEnhance.Brightness(image)
    return enhancer.enhance(brightness_factor)

In [30]:
def add_speckles(np_image):
    """Adds translucent elliptical speckles with random distribution."""
    speckle_mask = np.zeros_like(np_image, dtype=np.float32)
    speckle_mask = cv2.GaussianBlur(speckle_mask, (3, 3), sigmaX=1)

    speckle_density = random.randint(20, 500)
    distribution_type = random.choice(["uniform", "concentrated"])
    random_center_x = random.randint(np_image.shape[1] // 4, 3 * np_image.shape[1] // 4)
    random_center_y = random.randint(np_image.shape[0] // 4, 3 * np_image.shape[0] // 4)

    for _ in range(speckle_density):
        if distribution_type == "uniform":
            x = random.randint(0, np_image.shape[1] - 1)
            y = random.randint(0, np_image.shape[0] - 1)
        else:  # Concentrated
            spread_x = random.randint(100, 300)
            spread_y = random.randint(100, 300)
            x = random.randint(random_center_x - spread_x, random_center_x + spread_x)
            y = random.randint(random_center_y - spread_y, random_center_y + spread_y)
            x = np.clip(x, 0, np_image.shape[1] - 1)
            y = np.clip(y, 0, np_image.shape[0] - 1)

        size_x = random.randint(3, 6)
        size_y = random.randint(3, 6) # this would be used if we want the speckles to be elliptical
        intensity = random.uniform(25, 100)

        for i in range(-size_y, size_y):
            for j in range(-size_x, size_x):
                if ((j / size_x)**2 + (i / size_y)**2) <= 1:
                    if 0 <= y + i < np_image.shape[0] and 0 <= x + j < np_image.shape[1]:
                        speckle_mask[y + i, x + j] += intensity

    return np.clip(np_image + speckle_mask, 0, 255)

In [31]:
def apply_motion_blur(image, kernel_size=20, angle=0):
    """
    Applies motion blur to the input image in any direction.

    Parameters:
    - image (PIL.Image or NumPy array): The input image.
    - kernel_size (int): The size of the motion blur kernel. Higher values create stronger blur.
    - angle (float): The angle of the motion blur in degrees (0 = horizontal, 90 = vertical).

    Returns:
    - NumPy array: The motion-blurred image.
    """
    if isinstance(image, Image.Image):
        np_image = np.array(image)
    else:
        np_image = image

    # initial kernel (horizontal)
    kernel = np.zeros((kernel_size, kernel_size))
    kernel[kernel_size // 2, :] = np.ones(kernel_size)

    kernel /= kernel_size

    # rotate kernel
    rotation_matrix = cv2.getRotationMatrix2D((kernel_size // 2, kernel_size // 2), angle, 1)
    rotated_kernel = cv2.warpAffine(kernel, rotation_matrix, (kernel_size, kernel_size))

    blurred_image = cv2.filter2D(np_image, -1, rotated_kernel)

    return blurred_image

In [32]:
noising_functions_list = ['add_color_cast', 'add_gaussian_blur', 'add_gaussian_noise', 'adjust_brightness', 'add_speckles', 'apply_motion_blur']

### Encompassing function that applies the above underwater effects

In [33]:
def apply_random_underwater_effects(image):
    np_image = np.array(image).astype(np.float32)

    if random.random() < 0.9:
        np_image = add_color_cast(np_image)

    if random.random() < 0.8:
        np_image = add_gaussian_noise(np_image)

    if random.random() < 0.7:
        np_image = add_speckles(np_image)

    image = Image.fromarray(np_image.astype(np.uint8))

    if random.random() < 0.8:
        image = add_gaussian_blur(image)

    if random.random() < 0.7:
        image = adjust_brightness(image)

    if random.random() < 0.5:
        image = apply_motion_blur(image, kernel_size = int((random.random()*30) + 10), angle=random.random() * 90) #changed 
    
    return image

### Create Noised Images in root directory paths

In [34]:
root_dir = 'datasets/aquarium-data-cots/aquarium_pretrain' # Double check this is correct since adding to the VM

splits = ['train', 'test','valid']

RUN the cell below ONCE ONLY to create NOISY images. COMMENT OUT when done, there will be output. 

In [35]:
#RUN THIS CELL ONCE ONLY TO CREATE NOISY IMAGES. COMMENT OUT WHEN DONE. 
max_test_images = 10

for split in splits:
    images_dir = os.path.join(root_dir, split, 'images')
    noisy_images_dir = os.path.join(root_dir, split, 'noisy_images2')

    if not os.path.exists(noisy_images_dir):
        os.makedirs(noisy_images_dir)

    processed_count = 0

    for file_name in os.listdir(images_dir):
        if file_name.endswith('.jpg') or file_name.endswith('.png'):
            image_path = os.path.join(images_dir, file_name)
            image = Image.open(image_path).convert("RGB")

            noisy_image = apply_random_underwater_effects(image) # underwater effects applied here
            if isinstance(noisy_image, np.ndarray):
                noisy_image = Image.fromarray(noisy_image)

            noisy_image.save(os.path.join(noisy_images_dir, file_name))

            ''' Uncomment this section to test on a smaller batch of images '''
            # processed_count += 1
            # if processed_count >= max_test_images:
            #     break
        num_images = len(os.listdir(noisy_images_dir))    
    print(f"{num_images} Noisy images created for {split} images")

448 Noisy images created for train images
63 Noisy images created for test images
127 Noisy images created for valid images


### Visualize some images